In [ ]:
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
file_path = '/home/kkumari/PhD/data-analysis/fish-size-kinematics/fishSizeKin_hdf5_231128.mat'

In [ ]:
# Load the file to check its structure
with h5py.File(file_path, 'r') as file:
    # Listing all the items in the file
    items = list(file.items())

items

In [ ]:
# Function to convert HDF5 dataset to a numpy array
def hdf5_to_array(hdf5_dataset):
    if isinstance(hdf5_dataset, h5py.Dataset):  # Check if the item is a dataset
        return hdf5_dataset[...]  # Convert to numpy array
    return None

# Extracting data from 'fdata' and 'brs'
with h5py.File(file_path, 'r') as file:
    # Extracting 'fdata' group data
    fdata_group = file['fdata']
    fdata_data = {name: hdf5_to_array(dataset) for name, dataset in fdata_group.items() if hdf5_to_array(dataset) is not None}

    # Extracting 'brs' group data
    brs_group = fdata_group['brs']
    brs_data = {name: hdf5_to_array(dataset) for name, dataset in brs_group.items() if hdf5_to_array(dataset) is not None}

    #Extracting 'gld' group data
    gld_group = fdata_group['gld']
    gld_data = {name: hdf5_to_array(dataset) for name, dataset in gld_group.items() if hdf5_to_array(dataset) is not None}

# Creating a pandas DataFrame
fdata_df = pd.DataFrame(fdata_data)
brs_df = pd.DataFrame({name: data.squeeze() for name, data in brs_data.items()})
gld_df = pd.DataFrame({name: data.squeeze() for name, data in gld_data.items()})


fdata_df.head(), brs_df.head(), gld_df.head()   # Displaying the first few rows of each DataFrame


In [ ]:
brs_df.angle # Plotting the 'angle' column of 'brs_df'

In [ ]:
# Extracting unique fish identifiers
unique_fish = brs_df['fish'].unique()
unique_fish


In [ ]:
# Setting up the plot
plt.figure(figsize=(4, 2 * len(unique_fish)))
# Looping through each fish to plot their angle histogram
for i, fish in enumerate(unique_fish, 1):
    plt.subplot(len(unique_fish), 1, i)
    plt.hist(brs_df[brs_df['fish'] == fish]['angle'].dropna(), bins=100, alpha=0.7)
    plt.title(f'Histogram of Angles for Fish {fish}')
    plt.xlabel('Angle')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:
# Setting up the plot for radial histograms of angles
fig, axs = plt.subplots(1, len(unique_fish), figsize=(5 * len(unique_fish), 5), subplot_kw=dict(polar=True) if len(unique_fish) > 1 else {})

# Adjust axs to be a 1D array for uniform processing if there's only one fish
if len(unique_fish) == 1:
    axs = [axs]

# Looping through each fish to plot their angle histograms
for i, fish in enumerate(unique_fish):
    # Selecting the data for the current fish
    fish_data = brs_df[brs_df['fish'] == fish]

    # Plotting the radial histogram for angles
    ax = axs[i] if len(unique_fish) > 1 else axs[0]
    angle_data = fish_data['angle'].dropna()
    ax.hist(angle_data, bins=100, range=(-np.pi, np.pi), alpha=0.7)
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_title(f'Angles for Fish {fish}')

plt.tight_layout()
plt.show()


In [ ]:
# Creating 2D heatmaps of the XY plane for each fish
plt.figure(figsize=(5 * len(unique_fish), 5))

for i, fish in enumerate(unique_fish):
    plt.subplot(1, len(unique_fish), i + 1)
    
    # Selecting the data for the current fish
    fish_data = brs_df[brs_df['fish'] == fish]

    # Plotting the heatmap for XY coordinates
    plt.hexbin(fish_data['x'], fish_data['y'], gridsize=50, cmap='viridis', mincnt=1)
    plt.colorbar(label='Frequency')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'XY Trajectory Heatmap for Fish {fish}')

plt.tight_layout()
plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Creating 3D scatter plots for the trajectories of each fish
fig = plt.figure(figsize=(5 * len(unique_fish), 5))

for i, fish in enumerate(unique_fish):
    ax = fig.add_subplot(1, len(unique_fish), i + 1, projection='3d')
    
    # Selecting the data for the current fish
    fish_data = brs_df[brs_df['fish'] == fish]

    # Plotting the 3D scatter plot for XYZ coordinates
    scatter = ax.scatter(fish_data['x'], fish_data['y'], fish_data['z'], c=fish_data['z'], cmap='viridis', marker='.')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'3D Trajectory for Fish {fish}')
    fig.colorbar(scatter, ax=ax, label='Z-value')

plt.tight_layout()
plt.show()


In [ ]:
# Setting up the plot for stimuli type over time for each fish
plt.figure(figsize=(12, 3 * len(unique_fish)))

for i, fish in enumerate(unique_fish):
    plt.subplot(len(unique_fish), 1, i + 1)
    
    # Selecting the data for the current fish
    fish_data = brs_df[brs_df['fish'] == fish]

    # Plotting stimuli type over time
    plt.scatter(fish_data['ts'], fish_data['stim'], alpha=0.5, marker='.', s=10, c=fish_data['stim'], cmap='tab20')
    plt.title(f'Stimuli Type Over Time for Fish {fish}')
    plt.xlabel('Time')
    plt.ylabel('Stimuli Type')

plt.tight_layout()
plt.show()


In [ ]:
    # writing actual stim by adding correction factor
temp = fish_data['stim'] -1 
temp.unique()


# round to 2 decimal places

temp = temp.round(2)
temp.unique()

In [ ]:
# Setting up the plot for histograms of stimulus types for each fish
plt.figure(figsize=(6, 3 * len(unique_fish)))

for i, fish in enumerate(unique_fish):
    plt.subplot(len(unique_fish), 1, i + 1)
    
    # Selecting the data for the current fish
    fish_data = brs_df[brs_df['fish'] == fish]

    minSz = fish_data['stim'].min()
    maxSz = fish_data['stim'].max()
    step = 0.1

    mybins = np.arange(minSz , maxSz , step)

    # Plotting histogram of stimulus types
    plt.hist(temp.dropna(), alpha=0.7, edgecolor='black', bins=mybins)
    plt.title(f'Histogram of Stimulus Types for Fish {fish}')
    plt.xlabel('Stimulus Type')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:
# round mybins to 1 decimal place
mybins = np.round(mybins, 2)
mybins